<a href="https://colab.research.google.com/github/nlbtoan/AI-practis/blob/master/Example_ImageProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1I1hXBJX06qg8YI7F87xGRLKZCBE_aEDC', dest_path='./Image.zip', unzip=True)
!rm Image.zip

Unzipping...Done.


In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# 1. Histogram

In [ ]:
img = cv2.imread('Image/tiger.jpeg')

def showbgr(img):
    plt.imshow(img[...,[2,1,0]])
    plt.axis('off')
    plt.show()

def showgray(img):
    plt.imshow(img[...],cmap='gray')
    plt.axis('off')
    plt.show()

showbgr(img)

In [ ]:
def plot_hist(img, colors='bgr'):
    for i,col in enumerate(colors):
        histr = cv2.calcHist([img],[i],None,[256],[0,256])
        plt.plot(histr,color = col)
        plt.xlim([0,256])
        plt.ylim([0,600])
    plt.show()

plot_hist(img, 'bgr')

In [ ]:
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
showgray(gray)
equ = cv2.equalizeHist(gray)
showgray(equ)

In [ ]:
plot_hist(gray,'k')
plot_hist(equ,'k')

# Affine transformation


In [ ]:
img = cv2.imread('Image/meo.jpeg')
showbgr(img)
img.shape

In [ ]:
res = cv2.resize(img,None,fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
showbgr(res)
res.shape

In [ ]:
rows = img.shape[0]
cols = img.shape[1]
M    = np.float32([[1,0,100],[0,1,200]])
dst  = cv2.warpAffine(img,M,(cols,rows))
showbgr(dst)

# Blur and sharpen

In [ ]:
img = cv2.imread('Image/noise1.jpg')
showbgr(img)

In [ ]:
blur = cv2.blur(img,(5,5))
showbgr((blur))

In [ ]:
gauss_blur = cv2.GaussianBlur(img,(5,5),3)
showbgr((gauss_blur))

In [ ]:
blur = cv2.medianBlur(img,5)
showbgr((blur))

In [ ]:
bil_blur = cv2.bilateralFilter(img,5,75,75)
showbgr((bil_blur))

In [ ]:
kernel = np.array([[-1,-1,-1], 
                   [-1, 9,-1],
                   [-1,-1,-1]])
sharp = cv2.filter2D(blur, -1, kernel)
showbgr(sharp)

In [ ]:
kernel = np.array([[-1,-1,-1], 
                   [-1, 9,-1],
                   [-1,-1,-1]])
sharp = cv2.filter2D(blur, -1, kernel)
showbgr(sharp)

In [ ]:
sharp = cv2.addWeighted(blur, 1.5, gauss_blur, -0.5, 0)
showbgr(sharp)

# Brightness and contrast

In [ ]:
img = cv2.imread('Image/bright.jpeg')
showbgr(img)

In [ ]:
alpha = 1.4
beta = 100
res = np.clip((img*alpha + beta), 0,255).astype('uint8')
showbgr(res)

# Simple feature extraction

In [ ]:
img = cv2.imread('Image/meo.jpeg')
showbgr(img)

In [ ]:
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
showgray(gray)

In [ ]:
threshold = 127
ret,th1 = cv2.threshold(gray,threshold,255,cv2.THRESH_BINARY) 
showgray(th1)

In [ ]:
kernel = 11
# Apaptive thresh mean
th2 = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
        cv2.THRESH_BINARY,kernel,2) 
showgray(th2)

In [ ]:
kernel = 11
# Adaptive gaussian
th3 = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,kernel,2)
showgray(th3)

In [ ]:
plot_hist(th3,'k')

In [ ]:
uni, cnt = np.unique(th3, return_counts=True)
print(uni,cnt)

In [ ]:
# img = cv.imread('bird.jpg')
edged = cv2.Canny(th1, 80, 200)
showgray(edged)

In [ ]:
gray = cv2.imread('Image/Sodoku.jpeg',0)
showgray(gray)
blur = cv2.GaussianBlur(gray,(5,5),3)

In [ ]:
kernel = 11
th4 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY_INV,kernel,2)
showgray(th4)

In [ ]:
edges = cv2.Canny(th4,100,200)
plt.imshow(edges,'gray'),plt.axis('off')
plt.show()

In [ ]:
lines = cv2.HoughLines(th4,1,np.pi/180,230)
#Create a copy
img_copy=np.copy(gray)

for line in lines:
    rho,theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    cv2.line(img_copy,(x1,y1),(x2,y2),(0,0,255),2)
showgray(img_copy)